<a href="https://colab.research.google.com/github/Monunep/DPDPD/blob/main/DPDPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import random
import numpy as np
import copy

# ==============================================================================
#  CÁC HẰNG SỐ, DỮ LIỆU ĐẦU VÀO VÀ HÀM TIỆN ÍCH

# k cho các thuộc tính
NODE_TYPE = 0
TRUCK_DIST = 1
DRONE_DIST = 2
TRUCK_TIME = 3
DRONE_TIME = 4
A[i][j][k]
k

#  Node tp
TYPE_POT = 1
TYPE_HUB = 2
TYPE_CUSTOMER = 3

# (id, type, x, y, demand)
NODE_PROPERTIES = [
    {'id': 0, 'type': TYPE_POT,      'x': 50, 'y': 50, 'demand': 0},
    {'id': 1, 'type': TYPE_HUB,      'x': 20, 'y': 80, 'demand': 0},
    {'id': 2, 'type': TYPE_HUB,      'x': 80, 'y': 20, 'demand': 0},
    {'id': 3, 'type': TYPE_CUSTOMER, 'x': 15, 'y': 85, 'demand': 5},
    {'id': 4, 'type': TYPE_CUSTOMER, 'x': 95, 'y': 95, 'demand': 12},
    {'id': 5, 'type': TYPE_CUSTOMER, 'x': 50, 'y': 40, 'demand': 4},
    {'id': 6, 'type': TYPE_CUSTOMER, 'x': 85, 'y': 15, 'demand': 6} # Thêm một khách hàng
]

TRUCK_SPEED = 1.0
DRONE_SPEED = 10.0
DRONE_MAX_RANGE_DIST = 50
DRONE_CAPACITY = 10
TRUCK_CAPACITY = 50
DRONE_LAUNCH_TIME = 1
DRONE_LAND_TIME = 1
# random matrix
import random


def create_random_adjacency_matrix(num_nodes, num_hubs, num_customers,
                                   max_dist=100, truck_speed=1.0, drone_speed=2.0):
    """
    Tạo ra một ma trận kề 3D A[i][j][k] với các giá trị ngẫu nhiên.

    Args:
        num_nodes (int): Tổng số điểm (bao gồm kho).
        num_hubs (int): Số lượng trạm trung chuyển.
        num_customers (int): Số lượng khách hàng.
        max_dist (int): Khoảng cách tối đa có thể có giữa hai điểm.
        truck_speed (float): Tốc độ xe tải.
        drone_speed (float): Tốc độ drone.

    Returns:
        tuple: (Ma trận A, danh sách node_properties)
    """
    if 1 + num_hubs + num_customers != num_nodes:
        raise ValueError("Tổng số lượng các loại node khum khớp với num_nodes.")

    A = np.zeros((num_nodes, num_nodes, 5))
    node_properties = []

    #  ngẫu nhiên cho các đỉnh
    node_ids = list(range(1, num_nodes))
    random.shuffle(node_ids)

    # Node 0 là kho chính
    A[0, 0, NODE_TYPE] = TYPE_POT
    node_properties.append({'id': 0, 'type': TYPE_POT, 'demand': 0})

    # ngẫu nhiên các trạm trung chuyển
    for i in range(num_hubs):
        hub_id = node_ids.pop()
        A[hub_id, hub_id, NODE_TYPE] = TYPE_HUB
        node_properties.append({'id': hub_id, 'type': TYPE_HUB, 'demand': 0})

    # khách hàng
    for cust_id in node_ids:
        A[cust_id, cust_id, NODE_TYPE] = TYPE_CUSTOMER
        # Nhu cầa khách
        demand = random.randint(5, 20)
        node_properties.append({'id': cust_id, 'type': TYPE_CUSTOMER, 'demand': demand})

    # xếp node_properties theo ID
    node_properties.sort(key=lambda x: x['id'])

    # --- 2. Tạo thuộc tính ngẫu nhiên cho các cạnh ---
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j:
                continue

            # Tạo khoảng cách ngẫu nhiên
            # Không đối xứng, A[i,j]
            disconnect = random.random()
            dist = random.uniform(5, max_dist)
            if(disconnect>0.1):
              A[i, j, TRUCK_DIST] = dist
              A[i, j, DRONE_DIST] = dist
              A[i, j, TRUCK_TIME] = dist / truck_speed
              A[i, j, DRONE_TIME] = dist / drone_speed
            else:
              A[i, j, TRUCK_DIST] = 1000000
              A[i, j, DRONE_DIST] = 1000000
              A[i, j, TRUCK_TIME] = 1000000 / truck_speed
              A[i, j, DRONE_TIME] = 1000000 / drone_speed

    return A, node_properties
# Do Thi Euclid
def create_adjacency_matrix(nodes, truck_speed, drone_speed):
    """
    Tạo ma trận kề 3D A[i][j][k] từ danh sách các node.
    """
    num_nodes = len(nodes)
    A = np.zeros((num_nodes, num_nodes, 5))

    for i in range(num_nodes):
        A[i, i, NODE_TYPE] = nodes[i]['type']
        for j in range(num_nodes):
            if i == j: continue
            p1 = nodes[i]
            p2 = nodes[j]
            dist = math.sqrt((p1['x'] - p2['x'])**2 + (p1['y'] - p2['y'])**2)
            A[i, j, TRUCK_DIST] = dist
            A[i, j, DRONE_DIST] = dist
            A[i, j, TRUCK_TIME] = dist / truck_speed
            A[i, j, DRONE_TIME] = dist / drone_speed
    return A
def print_matrix_details(A, node_properties):
    """
    In ra một báo cáo chi tiết, dễ đọc về nội dung của ma trận kề A.
    """
    num_nodes = A.shape[0]
    node_props_map = {node['id']: node for node in node_properties}

    # Anh xa tu ma so sang ten node
    node_type_map = {
        TYPE_POT: "Depot",
        TYPE_HUB: "Hub",
        TYPE_CUSTOMER: "Customer"
    }

    print("\n" + "="*60)
    print(f"      VISUALIZATION OF ADJACENCY MATRIX A ({num_nodes}x{num_nodes}x{A.shape[2]})")
    print("="*60)

    # duong cheo
    print("\n--- 1. Node Properties (Diagonal A[i,i,:]) ---")
    for i in range(num_nodes):
        node_type_code = A[i, i, NODE_TYPE]
        node_type_str = node_type_map.get(node_type_code, "Unknown")
        demand = node_props_map[i]['demand']
        print(f"Node {i}: Type = {node_type_str} (code={int(node_type_code)}), Demand = {demand}")

    # i khac j

    print("\n--- 2. Edge Properties (Off-diagonal A[i,j,:]) ---")
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j:
                continue

            print(f"\n--- Edge {i} -> {j} ---")

            # matrix info
            truck_dist = A[i, j, TRUCK_DIST]
            truck_time = A[i, j, TRUCK_TIME]
            drone_dist = A[i, j, DRONE_DIST]
            drone_time = A[i, j, DRONE_TIME]

            print(f"  - Truck Info: (Distance: {truck_dist:.2f}, Time: {truck_time:.2f})")
            print(f"  - Drone Info: (Distance: {drone_dist:.2f}, Time: {drone_time:.2f})")

    print("\n" + "="*60)
    print("           END OF MATRIX VISUALIZATION")
    print("="*60 + "\n")

# ==============================================================================
# 2: INITIAL HEURISTIC
class MatrixSolver:
    def __init__(self, A, node_properties):
        self.A = A
        self.node_props = {node['id']: node for node in node_properties}
        self.num_nodes = A.shape[0]

    def initialize_solution(self):
        hub_ids = [i for i in range(self.num_nodes) if self.A[i, i, NODE_TYPE] == TYPE_HUB]
        customer_ids = [i for i in range(self.num_nodes) if self.A[i, i, NODE_TYPE] == TYPE_CUSTOMER]

        hub_drone_assignments = {hid: [] for hid in hub_ids}
        hub_demands = {hid: 0 for hid in hub_ids}
        unserved_customers = list(customer_ids)

        for req_id in list(unserved_customers):
            best_hub = min(hub_ids, key=lambda hid: self.A[req_id, hid, DRONE_DIST])
            min_dist = self.A[req_id, best_hub, DRONE_DIST]
            if min_dist * 2 <= DRONE_MAX_RANGE_DIST:
                hub_drone_assignments[best_hub].append(req_id)
                hub_demands[best_hub] += self.node_props[req_id]['demand']
                unserved_customers.remove(req_id)

        points_for_truck_ids = unserved_customers + [hid for hid in hub_ids if hub_demands[hid] > 0]
        truck_route = []
        current_loc = 0

        total_demand_for_truck = sum(self.node_props[pid]['demand'] for pid in unserved_customers) + sum(hub_demands.values())
        if total_demand_for_truck > TRUCK_CAPACITY:
            print("  - ERROR: Initial demand exceeds truck capacity. Solution may be infeasible.")

        while points_for_truck_ids:
            next_point_id = min(points_for_truck_ids, key=lambda pid: self.A[current_loc, pid, TRUCK_DIST])
            truck_route.append(next_point_id)
            current_loc = next_point_id
            points_for_truck_ids.remove(next_point_id)

        truck_launched_assignments = []

        return {
            'truck_route': truck_route,
            'hub_drone_assignments': hub_drone_assignments,
            'truck_launched_assignments': truck_launched_assignments,
            'unserved_requests': unserved_customers # Sẽ trống nếu tải trọng đủ
        }


#  3: SIÊU THUẬT TOÁN ALNS
class ALNSSolver:
    def __init__(self, A, node_properties, iterations=500, penalty_unserved=1000):
        self.A = A
        self.node_props = node_properties
        self.iterations = iterations
        self.penalty_unserved = penalty_unserved
        self.destroy_operators = [self.random_removal]
        self.repair_operators = [self.greedy_insertion]
        self.op_weights = {'destroy': [1] * len(self.destroy_operators), 'repair': [1] * len(self.repair_operators)}
        self.op_scores = {'destroy': [0]*len(self.destroy_operators), 'repair': [0]*len(self.repair_operators)}
        self.op_counts = {'destroy': [0]*len(self.destroy_operators), 'repair': [0]*len(self.repair_operators)}
        self.reaction_factor = 0.5
        self.start_temperature = 500
        self.end_temperature = 1
        self.cooling_rate = 0.995

    def solve(self, initial_solution):
        current_solution = copy.deepcopy(initial_solution)
        best_solution = copy.deepcopy(initial_solution)
        current_cost = self.calculate_cost(current_solution)
        best_cost = current_cost
        temperature = self.start_temperature
        print(f"Initial Cost: {best_cost:.2f}")

        for i in range(self.iterations):
            destroy_op_idx, destroy_op = self._select_operator('destroy')
            repair_op_idx, repair_op = self._select_operator('repair')

            partial_solution, removed_requests = destroy_op(current_solution)
            new_solution = repair_op(partial_solution, removed_requests)
            new_cost = self.calculate_cost(new_solution)

            score = 0
            if new_cost < best_cost:
                best_solution = copy.deepcopy(new_solution)
                best_cost = new_cost
                current_solution = copy.deepcopy(new_solution)
                current_cost = new_cost
                score = 3
                print(f"Iteration {i+1}/{self.iterations} | New Best Cost: {best_cost:.2f} (T={temperature:.2f})")
            elif new_cost < current_cost:
                current_solution = copy.deepcopy(new_solution)
                current_cost = new_cost
                score = 2
            elif random.random() < math.exp((current_cost - new_cost) / temperature):
                current_solution = copy.deepcopy(new_solution)
                current_cost = new_cost
                score = 1

            self._update_weights(destroy_op_idx, repair_op_idx, score)
            temperature = max(self.end_temperature, temperature * self.cooling_rate)

        print(f"\nFinished. Final Best Cost: {best_cost:.2f}")
        return best_solution

    def calculate_cost(self, solution):
        if not solution: return float('inf')
        total_dist = 0
        route = [0] + solution['truck_route'] + [0]
        for i in range(len(route) - 1):
            total_dist += self.A[route[i], route[i+1], TRUCK_DIST]
        for hub_id, reqs in solution['hub_drone_assignments'].items():
            for req_id in reqs:
                total_dist += 2 * self.A[hub_id, req_id, DRONE_DIST]
        for assign in solution['truck_launched_assignments']:
            total_dist += self.A[assign['launch'], assign['req_id'], DRONE_DIST] + self.A[assign['req_id'], assign['rendezvous'], DRONE_DIST]
        penalty = len(solution['unserved_requests']) * self.penalty_unserved
        return total_dist + penalty
# PHẦN THAY THẾ TRONG LỚP ALNSSolver

    # --- Các toán tử Phá hủy (Sửa lỗi nhỏ để đảm bảo không có unserved trùng lặp) ---
    def random_removal(self, solution, percentage=0.2):
        sol = copy.deepcopy(solution)
        served_customers = []

        # Lấy danh sách khách hàng đang được phục vụ
        truck_customers = {r for r in sol['truck_route'] if self.A[r,r,NODE_TYPE] == TYPE_CUSTOMER}
        hub_customers = {r for r_list in sol['hub_drone_assignments'].values() for r in r_list}
        truck_drone_customers = {a['req_id'] for a in sol['truck_launched_assignments']}

        served_customers = list(truck_customers | hub_customers | truck_drone_customers)

        if not served_customers:
            return sol, []

        num_to_remove = max(1, int(len(served_customers) * percentage))
        requests_to_remove = random.sample(served_customers, num_to_remove)

        # xay giải pháp bị phá hủy
        sol['truck_route'] = [r for r in sol['truck_route'] if r not in requests_to_remove]
        for hub_id in sol['hub_drone_assignments']:
            sol['hub_drone_assignments'][hub_id] = [r for r in sol['hub_drone_assignments'][hub_id] if r not in requests_to_remove]
        sol['truck_launched_assignments'] = [a for a in sol['truck_launched_assignments'] if a['req_id'] not in requests_to_remove]

        # dảm bảo danh sách unserved là duy nhất
        current_unserved = set(sol['unserved_requests'])
        current_unserved.update(requests_to_remove)
        sol['unserved_requests'] = list(current_unserved)

        return sol, requests_to_remove

    # greedy
    def greedy_insertion(self, partial_solution, requests_to_insert):
        """
        Chèn lại các khách hàng một cách an toàn và nhất quán.
        Đây là một heuristic tham lam đơn giản: nó chèn từng khách hàng một
        vào vị trí tốt nhất tại thời điểm đó.
        """
        sol = copy.deepcopy(partial_solution)

        # lặp qua các khách chèn lại
        for req_id in requests_to_insert:
            best_move = {'cost': float('inf'), 'solution': None}


            # 1. Thử chèn vào tuyến xe tải ở mọi vị trí
            for i in range(len(sol['truck_route']) + 1):
                temp_sol = copy.deepcopy(sol)
                temp_sol['truck_route'].insert(i, req_id)
                # Quan trọng: Xóa khỏi unserved trong bản sao tạm thời để tính chi phí đúng
                if req_id in temp_sol['unserved_requests']:
                    temp_sol['unserved_requests'].remove(req_id)

                cost = self.calculate_cost(temp_sol)
                if cost < best_move['cost']:
                    best_move['cost'] = cost
                    best_move['solution'] = temp_sol

            # 2. Thử gán cho mỗi trạm trung chuyển
            for hub_id in sol['hub_drone_assignments']:
                # (Trong hệ thống đầy đủ, cần kiểm tra ràng buộc tầm bay/tải trọng ở đây)
                temp_sol = copy.deepcopy(sol)
                temp_sol['hub_drone_assignments'][hub_id].append(req_id)
                if req_id in temp_sol['unserved_requests']:
                    temp_sol['unserved_requests'].remove(req_id)

                cost = self.calculate_cost(temp_sol)
                if cost < best_move['cost']:
                    best_move['cost'] = cost
                    best_move['solution'] = temp_sol

            # (Logic cho truck-launched drone  (idk hehe)

            #  nước đi tốt nhất cho req_id
            if best_move['solution']:
                sol = best_move['solution']

        return sol

    def _select_operator(self, op_type):
        weights = self.op_weights[op_type]
        total_weight = sum(weights)
        rand_val = random.uniform(0, total_weight)
        cumulative_weight = 0
        for i, weight in enumerate(weights):
            cumulative_weight += weight
            if rand_val <= cumulative_weight:
                return (i, self.destroy_operators[i]) if op_type == 'destroy' else (i, self.repair_operators[i])

    def _update_weights(self, destroy_idx, repair_idx, score):
        if score > 0: # Chỉ cập nhật nếu có sự thay đổi
            self.op_scores['destroy'][destroy_idx] += score
            self.op_scores['repair'][repair_idx] += score
            self.op_counts['destroy'][destroy_idx] += 1
            self.op_counts['repair'][repair_idx] += 1

            # Cập nhật định kỳ để tránh chia cho 0 và làm mới trọng số
            if sum(self.op_counts['destroy']) % 50 == 0:
                for op_type in ['destroy', 'repair']:
                    for i in range(len(self.op_weights[op_type])):
                        count = self.op_counts[op_type][i]
                        avg_score = self.op_scores[op_type][i] / count if count > 0 else 0
                        self.op_weights[op_type][i] = (1 - self.reaction_factor) * self.op_weights[op_type][i] + self.reaction_factor * avg_score

# 4: CHƯƠNG TRÌNH CHÍNH
if __name__ == "__main__":
    # input
    print("--- STEP 1: Creating Adjacency Matrix A ---")
    #euclid
    # A = create_adjacency_matrix(NODE_PROPERTIES, TRUCK_SPEED, DRONE_SPEED)
    print("--- STEP 1: Creating RANDOM Adjacency Matrix A ---")

    NUM_NODES = 50       # 1 kho + 20 trạm + 29 khách
    NUM_HUBS = 20
    NUM_CUSTOMERS = 29

    A, RANDOM_NODE_PROPERTIES = create_random_adjacency_matrix(
        num_nodes=NUM_NODES,
        num_hubs=NUM_HUBS,
        num_customers=NUM_CUSTOMERS
    )
    # GỌI HÀM IN
    print_matrix_details(A, RANDOM_NODE_PROPERTIES)
    #   heuristic ban đầu
    print("\n--- STEP 2: Creating Initial Heuristic Solution ---")
    initial_solver = MatrixSolver(A, RANDOM_NODE_PROPERTIES)
    initial_solution = initial_solver.initialize_solution()

    if initial_solution:
        #  Chạy ALNS để cải tiến giải pháp
        print("\n--- STEP 3: Running ALNS Metaheuristic ---")
        alns_solver = ALNSSolver(A, RANDOM_NODE_PROPERTIES, iterations=500, penalty_unserved=1000)
        final_solution = alns_solver.solve(initial_solution)

        # Final SOl
        print("\n" + "="*40)
        print("      FINAL SOLUTION SUMMARY")
        print("="*40)
        print("\n1. Truck Route:")
        if final_solution['truck_route']:
            path_str = ' -> '.join(map(str, final_solution['truck_route']))
            print(f"  - 0 -> {path_str} -> 0")
        else:
            print("  - Not used.")

        print("\n2. Hub-based Drone Deliveries:")
        for hub_id, reqs in final_solution['hub_drone_assignments'].items():
            if reqs:
                print(f"  - Hub {hub_id} serves: {reqs}")

        print("\n3. Truck-launched Drone Deliveries:")
        if final_solution['truck_launched_assignments']:
            for assign in final_solution['truck_launched_assignments']:
                print(f"  - Customer {assign['req_id']} from segment {assign['launch']}->{assign['rendezvous']}")
        else:
            print("  - None.")

        print("\n4. Unserved Requests:")
        if final_solution['unserved_requests']:
            print(f"  - {final_solution['unserved_requests']}")
        else:
            print("  - All requests served.")

Streaming output truncated to the last 5000 lines.
  - Drone Info: (Distance: 1000000.00, Time: 500000.00)

--- Edge 24 -> 39 ---
  - Truck Info: (Distance: 62.90, Time: 62.90)
  - Drone Info: (Distance: 62.90, Time: 31.45)

--- Edge 24 -> 40 ---
  - Truck Info: (Distance: 95.45, Time: 95.45)
  - Drone Info: (Distance: 95.45, Time: 47.73)

--- Edge 24 -> 41 ---
  - Truck Info: (Distance: 1000000.00, Time: 1000000.00)
  - Drone Info: (Distance: 1000000.00, Time: 500000.00)

--- Edge 24 -> 42 ---
  - Truck Info: (Distance: 71.61, Time: 71.61)
  - Drone Info: (Distance: 71.61, Time: 35.80)

--- Edge 24 -> 43 ---
  - Truck Info: (Distance: 40.90, Time: 40.90)
  - Drone Info: (Distance: 40.90, Time: 20.45)

--- Edge 24 -> 44 ---
  - Truck Info: (Distance: 89.96, Time: 89.96)
  - Drone Info: (Distance: 89.96, Time: 44.98)

--- Edge 24 -> 45 ---
  - Truck Info: (Distance: 5.67, Time: 5.67)
  - Drone Info: (Distance: 5.67, Time: 2.84)

--- Edge 24 -> 46 ---
  - Truck Info: (Distance: 37.31, Ti

In [ ]:
import math
import random
import numpy as np
import copy

# ==============================================================================
# SECTION 1: CÁC HẰNG SỐ VÀ THAM SỐ TOÀN CỤC
# ==============================================================================

# --- Chỉ số ma trận ---
NODE_TYPE, TRUCK_DIST, DRONE_DIST, TRUCK_TIME, DRONE_TIME = 0, 1, 2, 3, 4
# --- Loại Node ---
TYPE_POT, TYPE_HUB, TYPE_CUSTOMER = 1, 2, 3

# --- Tham số của Kịch bản (Scenario Parameters) ---
NUM_TRUCKS = 10
TRUCK_CAPACITY = 800
TRUCK_SPEED = 1.0

# --- Tham số Drone (Mô hình Giới hạn) ---
DRONES_PER_HUB = 4       # Mỗi trạm chỉ có thể phục vụ đồng thời 2 khách hàng
DRONES_PER_TRUCK = 2     # Mỗi xe tải chỉ có 1 drone để phóng đi
DRONE_CAPACITY = 50
DRONE_SPEED = 10.0
DRONE_MAX_RANGE_DIST = 150 # Tăng tầm bay cho bài toán lớn hơn
DRONE_LAUNCH_TIME = 1
DRONE_LAND_TIME = 1

# ==============================================================================
# SECTION 2: CÁC HÀM TẠO DỮ LIỆU
# ==============================================================================
# (Hàm create_random_adjacency_matrix và print_matrix_details giữ nguyên như cũ)
def create_random_adjacency_matrix(num_nodes, num_hubs, num_customers,
                                   max_dist=100, truck_speed=1.0, drone_speed=10.0):
    if 1 + num_hubs + num_customers != num_nodes:
        raise ValueError("Tổng số lượng các loại node không khớp với num_nodes.")
    A = np.zeros((num_nodes, num_nodes, 5))
    node_properties = []
    node_ids = list(range(1, num_nodes))
    random.shuffle(node_ids)
    A[0, 0, NODE_TYPE] = TYPE_POT
    node_properties.append({'id': 0, 'type': TYPE_POT, 'demand': 0})
    for i in range(num_hubs):
        hub_id = node_ids.pop()
        A[hub_id, hub_id, NODE_TYPE] = TYPE_HUB
        node_properties.append({'id': hub_id, 'type': TYPE_HUB, 'demand': 0})
    for cust_id in node_ids:
        A[cust_id, cust_id, NODE_TYPE] = TYPE_CUSTOMER
        demand = random.randint(5, 20)
        node_properties.append({'id': cust_id, 'type': TYPE_CUSTOMER, 'demand': demand})
    node_properties.sort(key=lambda x: x['id'])
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j:
                continue

            # Tạo khoảng cách ngẫu nhiên
            # Không đối xứng, A[i,j]
            disconnect = random.random()
            dist = random.uniform(5, max_dist)
            if(disconnect>0.1):
              A[i, j, TRUCK_DIST] = dist
              A[i, j, DRONE_DIST] = dist
              A[i, j, TRUCK_TIME] = dist / truck_speed
              A[i, j, DRONE_TIME] = dist / drone_speed
            else:
              A[i, j, TRUCK_DIST] = 1000000
              A[i, j, DRONE_DIST] = 1000000
              A[i, j, TRUCK_TIME] = 1000000 / truck_speed
              A[i, j, DRONE_TIME] = 1000000 / drone_speed

    return A, node_properties

def print_matrix_details(A, node_properties):
    """
    In ra một báo cáo chi tiết, dễ đọc về nội dung của ma trận kề A.
    """
    num_nodes = A.shape[0]
    node_props_map = {node['id']: node for node in node_properties}

    # Anh xa tu ma so sang ten node
    node_type_map = {
        TYPE_POT: "Depot",
        TYPE_HUB: "Hub",
        TYPE_CUSTOMER: "Customer"
    }

    print("\n" + "="*60)
    print(f"      VISUALIZATION OF ADJACENCY MATRIX A ({num_nodes}x{num_nodes}x{A.shape[2]})")
    print("="*60)

    # duong cheo
    print("\n--- 1. Node Properties (Diagonal A[i,i,:]) ---")
    for i in range(num_nodes):
        node_type_code = A[i, i, NODE_TYPE]
        node_type_str = node_type_map.get(node_type_code, "Unknown")
        demand = node_props_map[i]['demand']
        print(f"Node {i}: Type = {node_type_str} (code={int(node_type_code)}), Demand = {demand}")

    # i khac j

    print("\n--- 2. Edge Properties (Off-diagonal A[i,j,:]) ---")
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j:
                continue

            print(f"\n--- Edge {i} -> {j} ---")

            # matrix info
            truck_dist = A[i, j, TRUCK_DIST]
            truck_time = A[i, j, TRUCK_TIME]
            drone_dist = A[i, j, DRONE_DIST]
            drone_time = A[i, j, DRONE_TIME]

            print(f"  - Truck Info: (Distance: {truck_dist:.2f}, Time: {truck_time:.2f})")
            print(f"  - Drone Info: (Distance: {drone_dist:.2f}, Time: {drone_time:.2f})")

    print("\n" + "="*60)
    print("           END OF MATRIX VISUALIZATION")
    print("="*60 + "\n")

# HEURISTIC KHỞI TẠO CHO NHIỀU XE TẢI
class MultiVehicleMatrixSolver:
    def __init__(self, A, node_properties):
        self.A = A
        self.node_props = {node['id']: node for node in node_properties}
        self.num_nodes = A.shape[0]

    def initialize_solution(self):
        print("  - Building initial solution for MULTIPLE trucks...")
        hub_ids = [i for i in range(self.num_nodes) if self.A[i, i, NODE_TYPE] == TYPE_HUB]
        customer_ids = [i for i in range(self.num_nodes) if self.A[i, i, NODE_TYPE] == TYPE_CUSTOMER]

        hub_drone_assignments = {hid: [] for hid in hub_ids}
        hub_demands = {hid: 0 for hid in hub_ids}
        unserved_customers = list(customer_ids)

        # Giai đoạn 1: Gán drone từ trạm, có kiểm tra giới hạn
        for req_id in list(unserved_customers):
            # Sắp xếp các hub theo khoảng cách
            sorted_hubs = sorted(hub_ids, key=lambda hid: self.A[req_id, hid, DRONE_DIST])
            for hub_id in sorted_hubs:
                dist = self.A[req_id, hub_id, DRONE_DIST]
                # Kiểm tra cả tầm bay và số lượng drone có sẵn tại trạm
                if dist * 2 <= DRONE_MAX_RANGE_DIST and len(hub_drone_assignments[hub_id]) < DRONES_PER_HUB:
                    hub_drone_assignments[hub_id].append(req_id)
                    hub_demands[hub_id] += self.node_props[req_id]['demand']
                    unserved_customers.remove(req_id)
                    break # Chuyển sang khách hàng tiếp theo

        # Giai đoạn 2: Xây dựng tuần tự các tuyến xe tải
        points_for_truck_ids = unserved_customers + [hid for hid in hub_ids if hub_demands[hid] > 0]
        truck_routes = {f'truck_{i}': [] for i in range(NUM_TRUCKS)}

        for i in range(NUM_TRUCKS):
            truck_id = f'truck_{i}'
            current_loc = 0
            current_load = 0

            while points_for_truck_ids:
                # Tìm điểm gần nhất với vị trí hiện tại của xe tải
                best_point = min(points_for_truck_ids, key=lambda pid: self.A[current_loc, pid, TRUCK_DIST])
                demand_of_point = self.node_props[best_point]['demand'] if best_point in customer_ids else hub_demands[best_point]

                # Kiểm tra tải trọng
                if current_load + demand_of_point <= TRUCK_CAPACITY:
                    current_load += demand_of_point
                    truck_routes[truck_id].append(best_point)
                    current_loc = best_point
                    points_for_truck_ids.remove(best_point)
                else:
                    # Xe này đã đầy, chuyển sang xe tiếp theo
                    break

        # Những điểm còn lại sẽ không được phục vụ
        final_unserved = points_for_truck_ids

        return {
            'truck_routes': truck_routes,
            'hub_drone_assignments': hub_drone_assignments,
            'truck_launched_assignments': [],
            'unserved_requests': final_unserved
        }

# ==============================================================================
# SECTION 4: SIÊU THUẬT TOÁN ALNS (ĐÃ NÂNG CẤP)
# ==============================================================================
class ALNSSolver:
    # (Hàm __init__, _select_operator, _update_weights giữ nguyên)
    def __init__(self, A, node_properties, iterations=500, penalty_unserved=1000):
        self.A = A
        self.node_props = node_properties
        self.iterations = iterations
        self.penalty_unserved = penalty_unserved
        self.destroy_operators = [self.random_removal]
        self.repair_operators = [self.greedy_insertion]
        self.op_weights = {'destroy': [1] * len(self.destroy_operators), 'repair': [1] * len(self.repair_operators)}
        self.op_scores = {'destroy': [0]*len(self.destroy_operators), 'repair': [0]*len(self.repair_operators)}
        self.op_counts = {'destroy': [0]*len(self.destroy_operators), 'repair': [0]*len(self.repair_operators)}
        self.reaction_factor = 0.5
        self.start_temperature = 500
        self.end_temperature = 1
        self.cooling_rate = 0.995

    def solve(self, initial_solution):
        # (Hàm solve giữ nguyên, không cần thay đổi)
        current_solution = copy.deepcopy(initial_solution)
        best_solution = copy.deepcopy(initial_solution)
        current_cost = self.calculate_cost(current_solution)
        best_cost = current_cost
        temperature = self.start_temperature
        print(f"Initial Cost: {best_cost:.2f}")

        for i in range(self.iterations):
            destroy_op_idx, destroy_op = self._select_operator('destroy')
            repair_op_idx, repair_op = self._select_operator('repair')
            partial_solution, removed_requests = destroy_op(current_solution)
            new_solution = repair_op(partial_solution, removed_requests)
            new_cost = self.calculate_cost(new_solution)
            score = 0
            if new_cost < best_cost:
                best_solution = copy.deepcopy(new_solution)
                best_cost = new_cost
                current_solution = copy.deepcopy(new_solution)
                current_cost = new_cost
                score = 3
                print(f"Iteration {i+1}/{self.iterations} | New Best Cost: {best_cost:.2f} (T={temperature:.2f})")
            elif new_cost < current_cost:
                current_solution = copy.deepcopy(new_solution)
                current_cost = new_cost
                score = 2
            elif random.random() < math.exp((current_cost - new_cost) / temperature):
                current_solution = copy.deepcopy(new_solution)
                current_cost = new_cost
                score = 1
            self._update_weights(destroy_op_idx, repair_op_idx, score)
            temperature = max(self.end_temperature, temperature * self.cooling_rate)
        print(f"\nFinished. Final Best Cost: {best_cost:.2f}")
        return best_solution

    def calculate_cost(self, solution):
        """Đã cập nhật để tính chi phí cho nhiều xe tải."""
        if not solution: return float('inf')
        total_dist = 0
        # Chi phí của tất cả các xe tải
        for truck_id, route in solution['truck_routes'].items():
            route_with_depot = [0] + route + [0]
            for i in range(len(route_with_depot) - 1):
                total_dist += self.A[route_with_depot[i], route_with_depot[i+1], TRUCK_DIST]

        # (Phần còn lại giữ nguyên)
        for hub_id, reqs in solution['hub_drone_assignments'].items():
            for req_id in reqs:
                total_dist += 2 * self.A[hub_id, req_id, DRONE_DIST]
        for assign in solution['truck_launched_assignments']:
            total_dist += self.A[assign['launch'], assign['req_id'], DRONE_DIST] + self.A[assign['req_id'], assign['rendezvous'], DRONE_DIST]
        penalty = len(solution['unserved_requests']) * self.penalty_unserved
        return total_dist + penalty

    def random_removal(self, solution, percentage=0.3):
        """Đã cập nhật để làm việc với nhiều xe tải."""
        sol = copy.deepcopy(solution)
        served_customers = set()

        for route in sol['truck_routes'].values():
            for node in route:
                if self.A[node, node, NODE_TYPE] == TYPE_CUSTOMER:
                    served_customers.add(node)
        for reqs in sol['hub_drone_assignments'].values():
            served_customers.update(reqs)
        for assign in sol['truck_launched_assignments']:
            served_customers.add(assign['req_id'])

        if not served_customers: return sol, []

        num_to_remove = max(1, int(len(served_customers) * percentage))
        requests_to_remove = random.sample(list(served_customers), num_to_remove)

        for truck_id in sol['truck_routes']:
            sol['truck_routes'][truck_id] = [r for r in sol['truck_routes'][truck_id] if r not in requests_to_remove]
        for hub_id in sol['hub_drone_assignments']:
            sol['hub_drone_assignments'][hub_id] = [r for r in sol['hub_drone_assignments'][hub_id] if r not in requests_to_remove]
        sol['truck_launched_assignments'] = [a for a in sol['truck_launched_assignments'] if a['req_id'] not in requests_to_remove]

        current_unserved = set(sol['unserved_requests'])
        current_unserved.update(requests_to_remove)
        sol['unserved_requests'] = list(current_unserved)

        return sol, requests_to_remove

    def greedy_insertion(self, partial_solution, requests_to_insert):
        """Đã cập nhật để chèn vào nhiều xe và kiểm tra giới hạn drone."""
        sol = copy.deepcopy(partial_solution)

        for req_id in requests_to_insert:
            best_move = {'cost': float('inf'), 'solution': None}

            # 1. Thử chèn vào MỌI tuyến xe tải ở MỌI vị trí
            for truck_id, route in sol['truck_routes'].items():
                for i in range(len(route) + 1):
                    temp_sol = copy.deepcopy(sol)
                    temp_sol['truck_routes'][truck_id].insert(i, req_id)
                    if req_id in temp_sol['unserved_requests']:
                        temp_sol['unserved_requests'].remove(req_id)
                    cost = self.calculate_cost(temp_sol)
                    if cost < best_move['cost']:
                        best_move['cost'], best_move['solution'] = cost, temp_sol

            # 2. Thử gán cho MỌI trạm (có kiểm tra giới hạn drone)
            for hub_id in sol['hub_drone_assignments']:
                if len(sol['hub_drone_assignments'][hub_id]) < DRONES_PER_HUB:
                    temp_sol = copy.deepcopy(sol)
                    temp_sol['hub_drone_assignments'][hub_id].append(req_id)
                    if req_id in temp_sol['unserved_requests']:
                        temp_sol['unserved_requests'].remove(req_id)
                    cost = self.calculate_cost(temp_sol)
                    if cost < best_move['cost']:
                        best_move['cost'], best_move['solution'] = cost, temp_sol

            # (Logic cho truck-launched drone có thể thêm ở đây, có kiểm tra DRONES_PER_TRUCK)

            if best_move['solution']:
                sol = best_move['solution']

        return sol

    def _select_operator(self, op_type):
        # (Hàm này giữ nguyên)
        weights = self.op_weights[op_type]
        rand_val = random.uniform(0, sum(weights))
        cumulative_weight = 0
        for i, weight in enumerate(weights):
            cumulative_weight += weight
            if rand_val <= cumulative_weight:
                return (i, self.destroy_operators[i]) if op_type == 'destroy' else (i, self.repair_operators[i])

    def _update_weights(self, destroy_idx, repair_idx, score):
        # (Hàm này giữ nguyên)
        if score > 0:
            self.op_scores['destroy'][destroy_idx] += score
            self.op_scores['repair'][repair_idx] += score
            self.op_counts['destroy'][destroy_idx] += 1
            self.op_counts['repair'][repair_idx] += 1
            if sum(self.op_counts['destroy']) % 50 == 0:
                for op_type in ['destroy', 'repair']:
                    for i in range(len(self.op_weights[op_type])):
                        count = self.op_counts[op_type][i]
                        avg_score = self.op_scores[op_type][i] / count if count > 0 else 0
                        self.op_weights[op_type][i] = (1 - self.reaction_factor) * self.op_weights[op_type][i] + self.reaction_factor * avg_score

# ==============================================================================
# SECTION 5: CHƯƠNG TRÌNH CHÍNH
# ==============================================================================
if __name__ == "__main__":
    print("--- STEP 1: Creating RANDOM Adjacency Matrix A ---")
    NUM_TOTAL_NODES, NUM_HUBS, NUM_CUSTOMERS = 200, 5, 194

    A, RANDOM_NODE_PROPERTIES = create_random_adjacency_matrix(
        num_nodes=NUM_TOTAL_NODES, num_hubs=NUM_HUBS, num_customers=NUM_CUSTOMERS,
        max_dist=200
    )

    print("\n--- STEP 2: Creating Initial Heuristic Solution for Multiple Trucks ---")
    initial_solver = MultiVehicleMatrixSolver(A, RANDOM_NODE_PROPERTIES)
    initial_solution = initial_solver.initialize_solution()

    if initial_solution:
        print("\n--- STEP 3: Running ALNS Metaheuristic ---")
        alns_solver = ALNSSolver(A, RANDOM_NODE_PROPERTIES, iterations=50, penalty_unserved=5000)
        final_solution = alns_solver.solve(initial_solution)

        print("\n" + "="*40)
        print("      FINAL SOLUTION SUMMARY")
        print("="*40)
        print("\n1. Truck Routes:")
        for truck_id, route in final_solution['truck_routes'].items():
            if route:
                path_str = ' -> '.join(map(str, route))
                print(f"  - {truck_id}: 0 -> {path_str} -> 0")
            else:
                print(f"  - {truck_id}: Not used.")

        print("\n2. Hub-based Drone Deliveries:")
        for hub_id, reqs in final_solution['hub_drone_assignments'].items():
            if reqs:
                print(f"  - Hub {hub_id} serves: {reqs} ({len(reqs)}/{DRONES_PER_HUB} slots used)")

        # ... (phần in kết quả còn lại) ...
        print("\n4. Unserved Requests:")
        if final_solution['unserved_requests']:
            print(f"  - {final_solution['unserved_requests']}")
        else:
            print("  - All requests served.")

--- STEP 1: Creating RANDOM Adjacency Matrix A ---

--- STEP 2: Creating Initial Heuristic Solution for Multiple Trucks ---
  - Building initial solution for MULTIPLE trucks...

--- STEP 3: Running ALNS Metaheuristic ---
Initial Cost: 6005213.10
Iteration 1/50 | New Best Cost: 6004934.92 (T=500.00)
Iteration 2/50 | New Best Cost: 4004765.67 (T=497.50)
Iteration 3/50 | New Best Cost: 2004773.56 (T=495.01)
Iteration 4/50 | New Best Cost: 2004731.30 (T=492.54)
Iteration 5/50 | New Best Cost: 2004667.73 (T=490.07)
Iteration 7/50 | New Best Cost: 2004494.20 (T=485.19)
Iteration 8/50 | New Best Cost: 4097.87 (T=482.76)
Iteration 36/50 | New Best Cost: 4066.74 (T=419.54)
Iteration 38/50 | New Best Cost: 4037.17 (T=415.36)
Iteration 43/50 | New Best Cost: 4007.52 (T=405.08)
Iteration 46/50 | New Best Cost: 3842.63 (T=399.03)

Finished. Final Best Cost: 3842.63

      FINAL SOLUTION SUMMARY

1. Truck Routes:
  - truck_0: 0 -> 197 -> 11 -> 79 -> 78 -> 20 -> 199 -> 191 -> 127 -> 90 -> 153 -> 104 

In [18]:
import math
import random
import numpy as np
import copy

# ==============================================================================
# SECTION 1: CÁC HẰNG SỐ VÀ THAM SỐ TOÀN CỤC
# ==============================================================================

# --- Chỉ số ma trận & Loại Node ---
NODE_TYPE, TRUCK_DIST, DRONE_DIST, TRUCK_TIME, DRONE_TIME = 0, 1, 2, 3, 4
TYPE_POT, TYPE_HUB, TYPE_CUSTOMER = 1, 2, 3

# --- Tham số của Kịch bản (Scenario Parameters) ---
NUM_TRUCKS = 10
TRUCK_CAPACITY = 100
TRUCK_SPEED = 1.0

# --- Tham số Drone (Mô hình Giới hạn) ---
DRONES_PER_HUB = 2
DRONES_PER_TRUCK = 1
DRONE_CAPACITY = 1000
DRONE_SPEED = 100.0
DRONE_MAX_RANGE_DIST = 150
SERVICE_TIME = 5 # Thời gian dỡ hàng/phục vụ tại mỗi điểm dừng

# ==============================================================================
# SECTION 2: CÁC HÀM TẠO DỮ LIỆU
# ==============================================================================

def create_random_adjacency_matrix(num_nodes, num_hubs, num_customers,
                                   max_dist=100, truck_speed=1.0, drone_speed=2.0):
    if 1 + num_hubs + num_customers != num_nodes:
        raise ValueError("Tổng số lượng các loại node không khớp với num_nodes.")

    A = np.zeros((num_nodes, num_nodes, 5))
    node_properties = []

    node_ids = list(range(1, num_nodes))
    random.shuffle(node_ids)

    A[0, 0, NODE_TYPE] = TYPE_POT
    node_properties.append({'id': 0, 'type': TYPE_POT, 'demand': 0, 'ready_time': 0, 'due_date': 2000})

    for i in range(num_hubs):
        hub_id = node_ids.pop()
        A[hub_id, hub_id, NODE_TYPE] = TYPE_HUB
        node_properties.append({'id': hub_id, 'type': TYPE_HUB, 'demand': 0, 'ready_time': 0, 'due_date': float('inf')})

    for cust_id in node_ids:
        A[cust_id, cust_id, NODE_TYPE] = TYPE_CUSTOMER
        demand = random.randint(5, 15)
        ready_time = random.randint(0, 8000)
        due_date = ready_time + random.randint(100, 3000)
        node_properties.append({'id': cust_id, 'type': TYPE_CUSTOMER, 'demand': demand,
                                'ready_time': ready_time, 'due_date': due_date})

    node_properties.sort(key=lambda x: x['id'])

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j: continue
            dist = random.uniform(10, max_dist)
            A[i, j, TRUCK_DIST], A[i, j, DRONE_DIST] = dist, dist
            A[i, j, TRUCK_TIME], A[i, j, DRONE_TIME] = dist / truck_speed, dist / drone_speed

    return A, node_properties

# ==============================================================================
# SECTION 3: HEURISTIC KHỞI TẠO
# ==============================================================================
class MultiVehicleMatrixSolver:
    def __init__(self, A, node_properties):
        self.A = A
        self.node_props = {node['id']: node for node in node_properties}
        self.num_nodes = A.shape[0]

    def initialize_solution(self):
        # ... (logic heuristic đơn giản, ALNS sẽ làm phần việc chính)
        customer_ids = [i for i in range(self.num_nodes) if self.A[i, i, NODE_TYPE] == TYPE_CUSTOMER]
        truck_routes = {f'truck_{i}': [] for i in range(NUM_TRUCKS)}

        # Phân bổ khách hàng đều cho các xe tải
        for i, cust_id in enumerate(customer_ids):
            truck_index = i % NUM_TRUCKS
            truck_routes[f'truck_{truck_index}'].append(cust_id)

        return {
            'truck_routes': truck_routes, 'hub_drone_assignments': {},
            'truck_launched_assignments': [], 'unserved_requests': []
        }

# ==============================================================================
# SECTION 4: SIÊU THUẬT TOÁN ALNS (PHIÊN BẢN HOÀN CHỈNH)
# ==============================================================================
class ALNSSolver:
    def __init__(self, A, node_properties, iterations=1000, penalty_unserved=5000):
        self.A = A
        self.node_props = node_properties
        self.iterations = iterations
        self.penalty_unserved = penalty_unserved
        self.destroy_operators = [self.random_removal] # Có thể thêm Shaw, etc.
        self.repair_operators = [self.greedy_insertion] # Có thể thêm Regret, etc.
        self.op_weights = {'destroy': [1], 'repair': [1]}
        self.op_scores = {'destroy': [0], 'repair': [0]}
        self.op_counts = {'destroy': [0], 'repair': [0]}
        self.reaction_factor = 0.5
        self.start_temperature = 100
        self.end_temperature = 0.1
        self.cooling_rate = 0.999

    # --- HÀM ĐÁNH GIÁ VÀ TÍNH CHI PHÍ TOÀN DIỆN ---
    def calculate_cost(self, solution):
        total_cost = 0

        # 1. Chi phí phạt cho khách hàng không được phục vụ
        total_cost += len(solution['unserved_requests']) * self.penalty_unserved

        # 2. Đánh giá các tuyến xe tải và các nhiệm vụ drone liên quan
        all_hub_assignments = solution['hub_drone_assignments']
        all_truck_drone_assignments = solution['truck_launched_assignments']

        for truck_id, route in solution['truck_routes'].items():
            current_time = 0.0
            current_load = 0.0
            last_node = 0

            # Kiểm tra tải trọng trước
            route_demand = sum(self.node_props[node_id]['demand'] for node_id in route)
            if route_demand > TRUCK_CAPACITY: return float('inf')

            route_with_depot = [0] + route + [0]

            for i in range(len(route_with_depot) - 1):
                start_node = route_with_depot[i]
                end_node = route_with_depot[i+1]

                # Thời gian khởi hành của xe tải từ start_node
                departure_time_from_start = current_time

                # Chi phí di chuyển của xe tải trên chặng này
                truck_travel_time = self.A[start_node, end_node, TRUCK_TIME]
                truck_travel_dist = self.A[start_node, end_node, TRUCK_DIST]
                total_cost += truck_travel_dist

                # --- KIỂM TRA DRONE TỪ XE TẢI TRÊN CHẶNG NÀY ---
                missions_on_this_leg = [m for m in all_truck_drone_assignments
                                        if m.get('truck_id') == truck_id and
                                           m.get('launch') == start_node and
                                           m.get('rendezvous') == end_node]

                if len(missions_on_this_leg) > DRONES_PER_TRUCK: return float('inf')

                for mission in missions_on_this_leg:
                    cust_id = mission['req_id']
                    drone_flight_time = self.A[start_node, cust_id, DRONE_TIME] + self.A[cust_id, end_node, DRONE_TIME]
                    drone_total_time = drone_flight_time # Giả sử launch/land time = 0 để đơn giản

                    if drone_total_time > truck_travel_time: return float('inf') # Lỗi đồng bộ hóa

                    drone_arrival_at_cust = departure_time_from_start + self.A[start_node, cust_id, DRONE_TIME]
                    cust_info = self.node_props[cust_id]

                    drone_service_start = max(drone_arrival_at_cust, cust_info['ready_time'])
                    if drone_service_start > cust_info['due_date']: return float('inf') # Lỗi time window

                    total_cost += self.A[start_node, cust_id, DRONE_DIST] + self.A[cust_id, end_node, DRONE_DIST]

                # Cập nhật thời gian của xe tải khi đến end_node
                current_time += truck_travel_time
                end_node_info = self.node_props[end_node]
                current_time = max(current_time, end_node_info['ready_time']) # Chờ đợi

                if current_time > end_node_info['due_date']: return float('inf') # Lỗi time window xe tải

                current_time += SERVICE_TIME # Thời gian phục vụ của xe tải

                # --- KIỂM TRA DRONE TỪ TRẠM (NẾU end_node LÀ TRẠM) ---
                if end_node_info['type'] == TYPE_HUB:
                    assigned_customers = all_hub_assignments.get(end_node, [])
                    if len(assigned_customers) > DRONES_PER_HUB: return float('inf')

                    # Thời gian drone có thể bắt đầu bay từ trạm
                    drone_departure_from_hub = current_time

                    for cust_id in assigned_customers:
                        drone_arrival_at_cust = drone_departure_from_hub + self.A[end_node, cust_id, DRONE_TIME]
                        cust_info = self.node_props[cust_id]

                        drone_service_start = max(drone_arrival_at_cust, cust_info['ready_time'])
                        if drone_service_start > cust_info['due_date']: return float('inf')

                        total_cost += 2 * self.A[end_node, cust_id, DRONE_DIST]

                last_node = end_node

        return total_cost

    def solve(self, initial_solution):
        # ... (logic vòng lặp chính không đổi)
        current_solution = copy.deepcopy(initial_solution)
        best_solution = copy.deepcopy(initial_solution)
        current_cost = self.calculate_cost(current_solution)
        best_cost = current_cost
        temperature = self.start_temperature
        print(f"Initial Cost: {best_cost:.2f} (May be infeasible)")

        for i in range(self.iterations):
            destroy_op_idx, destroy_op = self._select_operator('destroy')
            repair_op_idx, repair_op = self._select_operator('repair')
            partial_solution, removed_requests = destroy_op(current_solution)
            new_solution = repair_op(partial_solution, removed_requests)
            new_cost = self.calculate_cost(new_solution)
            score = 0
            if new_cost < best_cost:
                best_solution, best_cost = copy.deepcopy(new_solution), new_cost
                current_solution, current_cost = copy.deepcopy(new_solution), new_cost
                score = 3
                print(f"Iteration {i+1}/{self.iterations} | New Best Cost: {best_cost:.2f} (T={temperature:.2f})")
            elif new_cost < current_cost:
                current_solution, current_cost = copy.deepcopy(new_solution), new_cost
                score = 2
            elif random.random() < math.exp((current_cost - new_cost) / temperature):
                current_solution, current_cost = copy.deepcopy(new_solution), new_cost
                score = 1
            self._update_weights(destroy_op_idx, repair_op_idx, score)
            temperature = max(self.end_temperature, temperature * self.cooling_rate)
        print(f"\nFinished. Final Best Cost: {best_cost:.2f}")
        return best_solution

    def random_removal(self, solution, percentage=0.3):
        # ... (logic không đổi)
        sol = copy.deepcopy(solution)
        served_customers = set()
        for route in sol['truck_routes'].values():
            for node in route:
                if self.A[node, node, NODE_TYPE] == TYPE_CUSTOMER:
                    served_customers.add(node)
        for reqs in sol['hub_drone_assignments'].values(): served_customers.update(reqs)
        for assign in sol['truck_launched_assignments']: served_customers.add(assign['req_id'])
        if not served_customers: return sol, []
        num_to_remove = max(1, int(len(served_customers) * percentage))
        requests_to_remove = random.sample(list(served_customers), num_to_remove)
        for truck_id in sol['truck_routes']:
            sol['truck_routes'][truck_id] = [r for r in sol['truck_routes'][truck_id] if r not in requests_to_remove]
        for hub_id in sol['hub_drone_assignments']:
            sol['hub_drone_assignments'][hub_id] = [r for r in sol['hub_drone_assignments'][hub_id] if r not in requests_to_remove]
        sol['truck_launched_assignments'] = [a for a in sol['truck_launched_assignments'] if a['req_id'] not in requests_to_remove]
        current_unserved = set(sol['unserved_requests'])
        current_unserved.update(requests_to_remove)
        sol['unserved_requests'] = list(current_unserved)
        return sol, requests_to_remove

    def greedy_insertion(self, partial_solution, requests_to_insert):
        sol = copy.deepcopy(partial_solution)
        for req_id in requests_to_insert:
            best_move = {'cost': float('inf'), 'solution': None}
            # 1. Thử chèn vào tuyến xe tải
            for truck_id, route in sol['truck_routes'].items():
                for i in range(len(route) + 1):
                    temp_sol = copy.deepcopy(sol)
                    temp_sol['truck_routes'][truck_id].insert(i, req_id)
                    temp_sol['unserved_requests'].remove(req_id)
                    cost = self.calculate_cost(temp_sol)
                    if cost < best_move['cost']: best_move['cost'], best_move['solution'] = cost, temp_sol

            # 2. Thử gán cho trạm
            for hub_id in sol['hub_drone_assignments']:
                temp_sol = copy.deepcopy(sol)
                temp_sol['hub_drone_assignments'][hub_id].append(req_id)
                temp_sol['unserved_requests'].remove(req_id)
                cost = self.calculate_cost(temp_sol)
                if cost < best_move['cost']: best_move['cost'], best_move['solution'] = cost, temp_sol

            # 3. Thử gán cho drone từ xe tải
            for truck_id, route in sol['truck_routes'].items():
                route_with_depot = [0] + route + [0]
                for i in range(len(route_with_depot) - 1):
                    temp_sol = copy.deepcopy(sol)
                    mission = {'req_id': req_id, 'truck_id': truck_id,
                               'launch': route_with_depot[i], 'rendezvous': route_with_depot[i+1]}
                    temp_sol['truck_launched_assignments'].append(mission)
                    temp_sol['unserved_requests'].remove(req_id)
                    cost = self.calculate_cost(temp_sol)
                    if cost < best_move['cost']: best_move['cost'], best_move['solution'] = cost, temp_sol

            if best_move['solution']: sol = best_move['solution']
        return sol

    def _select_operator(self, op_type):
        # ... (logic không đổi)
        weights = self.op_weights[op_type]
        rand_val = random.uniform(0, sum(weights))
        cumulative_weight = 0
        for i, weight in enumerate(weights):
            cumulative_weight += weight
            if rand_val <= cumulative_weight:
                return (i, self.destroy_operators[i]) if op_type == 'destroy' else (i, self.repair_operators[i])

    def _update_weights(self, destroy_idx, repair_idx, score):
        # ... (logic không đổi)
        if score > 0:
            self.op_scores['destroy'][destroy_idx] += score
            self.op_scores['repair'][repair_idx] += score
            self.op_counts['destroy'][destroy_idx] += 1
            self.op_counts['repair'][repair_idx] += 1
            if sum(self.op_counts['destroy']) % 50 == 0:
                for op_type in ['destroy', 'repair']:
                    for i in range(len(self.op_weights[op_type])):
                        count = self.op_counts[op_type][i]
                        avg_score = self.op_scores[op_type][i] / count if count > 0 else 0
                        self.op_weights[op_type][i] = (1 - self.reaction_factor) * self.op_weights[op_type][i] + self.reaction_factor * avg_score

# ==============================================================================
# SECTION 5: CHƯƠNG TRÌNH CHÍNH
# ==============================================================================
if __name__ == "__main__":
    print("--- STEP 1: Creating RANDOM Adjacency Matrix with Time Windows ---")
    NUM_TOTAL_NODES, NUM_HUBS, NUM_CUSTOMERS = 30, 5, 24

    A, RANDOM_NODE_PROPERTIES = create_random_adjacency_matrix(
        num_nodes=NUM_TOTAL_NODES, num_hubs=NUM_HUBS, num_customers=NUM_CUSTOMERS, max_dist=200
    )

    print("\n--- STEP 2: Creating Initial Solution (may be infeasible) ---")
    initial_solver = MultiVehicleMatrixSolver(A, RANDOM_NODE_PROPERTIES)
    initial_solution = initial_solver.initialize_solution()

    if initial_solution:
        print("\n--- STEP 3: Running ALNS Metaheuristic (handling all constraints) ---")
        alns_solver = ALNSSolver(A, RANDOM_NODE_PROPERTIES, iterations=500, penalty_unserved=10000)
        final_solution = alns_solver.solve(initial_solution)

        print("\n" + "="*50)
        print(" " * 15 + "FINAL SOLUTION SUMMARY")
        print("="*50)
        print("\n1. Truck Routes:")
        for truck_id, route in final_solution['truck_routes'].items():
            if route: print(f"  - {truck_id}: 0 -> {' -> '.join(map(str, route))} -> 0")
            else: print(f"  - {truck_id}: Not used.")

        print("\n2. Hub-based Drone Deliveries:")
        for hub_id, reqs in final_solution['hub_drone_assignments'].items():
            if reqs: print(f"  - Hub {hub_id} serves: {reqs} ({len(reqs)}/{DRONES_PER_HUB} slots used)")

        print("\n3. Truck-launched Drone Deliveries:")
        if final_solution['truck_launched_assignments']:
            for assign in final_solution['truck_launched_assignments']:
                print(f"  - Cust {assign['req_id']} from {assign['truck_id']} on segment {assign['launch']}->{assign['rendezvous']}")
        else: print("  - None.")

        print("\n4. Unserved Requests:")
        if final_solution['unserved_requests']: print(f"  - {final_solution['unserved_requests']}")
        else: print("  - All requests served.")
        print("="*50)

--- STEP 1: Creating RANDOM Adjacency Matrix with Time Windows ---

--- STEP 2: Creating Initial Solution (may be infeasible) ---

--- STEP 3: Running ALNS Metaheuristic (handling all constraints) ---
Initial Cost: inf (May be infeasible)

Finished. Final Best Cost: inf

               FINAL SOLUTION SUMMARY

1. Truck Routes:
  - truck_0: 0 -> 3 -> 14 -> 26 -> 0
  - truck_1: 0 -> 4 -> 15 -> 27 -> 0
  - truck_2: 0 -> 5 -> 17 -> 28 -> 0
  - truck_3: 0 -> 6 -> 18 -> 29 -> 0
  - truck_4: 0 -> 7 -> 19 -> 0
  - truck_5: 0 -> 8 -> 20 -> 0
  - truck_6: 0 -> 10 -> 21 -> 0
  - truck_7: 0 -> 11 -> 23 -> 0
  - truck_8: 0 -> 12 -> 24 -> 0
  - truck_9: 0 -> 13 -> 25 -> 0

2. Hub-based Drone Deliveries:

3. Truck-launched Drone Deliveries:
  - None.

4. Unserved Requests:
  - All requests served.
